<h1><center>Bioptim Workshop</center></h1>

# Let's go for a multiphase problem !
In this exercice, you will implement a vertical 2D jumper that must reach a height of 1 meter with one torque-actuated leg made of two segments.
The vertical degree of freedom actuation must be forced to zero, to mimic free flying conditions.
The lower segment of the leg must be constrained with a contact point in the first phase which will then be released in the second phase.
The main objective is to minimize torques during each phase.
The first phase will last for 0.2s with 40 nodes and the second will last for 0.4s with 40 nodes.

The motion will look like this :

![SegmentLocal](img/slider.gif)

In [ ]:
from bioptim import *

First, inspect the model ("models/Slider1Leg.bioMod") to understand how the contact point is implemented. We will use it during the first phase, to push on the ground before takeoff !

In [ ]:
model = (
    TorqueBiorbdModel("models/Slider1Leg.bioMod", contact_types=[ContactType.RIGID_EXPLICIT]), 
    TorqueBiorbdModel("models/Slider1Leg.bioMod")
)
nq = model[0].nb_q
ntau = model[0].nb_tau

# This is your turn
...
tau_min, tau_max, tau_init = -200, 200, 0

# State path constraints
x_bounds = BoundsList()
x_bounds.add("q", bounds=model[0].bounds_from_ranges("q"), phase=0)
x_bounds.add("qdot", bounds=model[0].bounds_from_ranges("qdot"), phase=0)
x_bounds.add("q", bounds=model[1].bounds_from_ranges("q"), phase=1)
x_bounds.add("qdot", bounds=model[1].bounds_from_ranges("qdot"), phase=1)

x_bounds[0]["q"][:, 0] = [0, 3*np.pi / 8, -3*np.pi / 4]  # Start in demi-squat pose
x_bounds[0]["q"].min[0, -1] = 0  # Translation of the slider at end between 0...
x_bounds[0]["q"].max[0, -1] = 0.25  # ... and 0.25 m
x_bounds[0]["qdot"][:, 0] = 0  # Start at rest

x_bounds[1]["q"][0, -1] = 1  # End simulation at 1 m of vertical translation
x_bounds[1]["qdot"][0, -1] = 0  # without speed

# Give some limits to the torque
u_bounds = BoundsList()
u_bounds.add("tau", min_bound=[tau_min] * ntau, max_bound=[tau_max] * ntau, phase=0)
u_bounds.add("tau", min_bound=[tau_min] * ntau, max_bound=[tau_max] * ntau, phase=1)

# But remove the capability to actuate the translational joint (first DOF)
u_bounds[0]["tau"][0, :] = 0
u_bounds[1]["tau"][0, :] = 0

# To be continued
...